In [2]:
from pathlib import Path
import pandas as pd
import os
import csv
import numpy as np
import requests
import time
import io
import yfinance as yf
import warnings
# Initial imports
#%tensorflow_version 1.x
import tensorflow
import mibian
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lstm
import psutil
psutil.virtual_memory()

svmem(total=17073303552, available=3228450816, percent=81.1, used=13844852736, free=3228450816)

In [ ]:
#I am importing the lstm python file which will allow me to use its functions

import lstm
#this is how you use its functions
#results = lstm.main_lstm('features.csv')

In [3]:
#This is the etf sector df
df = pd.read_pickle('etf.pickle')
df = df.set_index(pd.to_datetime(df.DataDate, infer_datetime_format=True)) 
df

,optionkey,Symbol,ExpirationDate,AskPrice,AskSize,BidPrice,BidSize,LastPrice,PutCall,StrikePrice,Volume,openinterest,UnderlyingPrice,DataDate
DataDate,,,,,,,,,,,,,,
2017-03-22,MTUM2017-04-21c80.002017-03-22,MTUM,2017-04-21,3.70,5,1.50,5,0.00,call,80.00,0,0,82.34,2017-03-22
2017-03-22,MTUM2017-04-21p80.002017-03-22,MTUM,2017-04-21,0.70,1,0.00,0,0.00,put,80.00,0,0,82.34,2017-03-22
2017-03-22,MTUM2017-04-21c81.002017-03-22,MTUM,2017-04-21,2.90,5,0.65,5,0.00,call,81.00,0,0,82.34,2017-03-22
2017-03-22,MTUM2017-04-21p81.002017-03-22,MTUM,2017-04-21,1.70,5,0.00,0,0.00,put,81.00,0,0,82.34,2017-03-22
2017-03-22,MTUM2017-04-21c82.002017-03-22,MTUM,2017-04-21,2.15,5,0.00,0,0.00,call,82.00,0,0,82.34,2017-03-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-30,VOO2020-02-21P345.002019-12-30,VOO,2020-02-21,50.90,24,49.50,11,0,put,345.00,0,0,295.0400,2019-12-30
2019-12-30,VOO2020-02-21P350.002019-12-30,VOO,2020-02-21,55.80,25,54.50,22,0,put,350.00,0,0,295.0400,2019-12-30
2019-12-30,VOO2020-02-21P355.002019-12-30,VOO,2020-02-21,60.60,11,59.60,11,0,put,355.00,0,0,295.0400,2019-12-30


In [4]:
##Clue to whats going wrong
# df = df.drop(columns=['DataDate'])
df['DataDate'].dtype

dtype('O')

In [19]:
start_date = '2016-01-01'
end_date = '2020-12-31'
symbol_list = ['MTUM', 'SPY', 'NDAQ', 'VOO' ]
####Get the underlying stock prices and volume by date

def get_underlying(Symbols):
# create empty dataframe
    stock_final = pd.DataFrame()
# iterate over each symbol
    for i in Symbols:  


        try:
            # download the stock price 
            stock = []
            stock = yf.download(i,start=start_date, end=end_date, progress=False)

            # append the individual stock prices 
            if len(stock) == 0:
                None
            else:
                stock['Name']=i
                stock_final = stock_final.append(stock,sort=False)
                stock_final= stock_final.drop(['Open', 'High', 'Low', 'Adj Close'], axis=1)
        except Exception:
            None
    return stock_final

underlying = get_underlying(symbol_list)
underlying

,Close,Volume,Name
Date,,,
2015-12-31,73.290001,178800,MTUM
2016-01-04,71.809998,494400,MTUM
2016-01-05,72.150002,319800,MTUM
2016-01-06,71.769997,262600,MTUM
2016-01-07,70.169998,212000,MTUM
...,...,...,...
2020-12-23,337.809998,3682200,VOO
2020-12-24,339.160004,1447500,VOO
2020-12-28,342.109985,2798800,VOO


In [7]:
### Run the function cell below to use this function
##This function works
tf = feature_engineering(df)
tf

,optionkey,Symbol,ExpirationDate,AskPrice,AskSize,BidPrice,BidSize,LastPrice,PutCall,StrikePrice,Volume,openinterest,UnderlyingPrice,time delta,% out,exp?,DateKey
DataDate,,,,,,,,,,,,,,,,,
2017-03-22,MTUM2017-04-21c80.002017-03-22,MTUM,2017-04-21,3.70,5,1.50,5,0.00,call,80.0,0,0,82.34,30 days,-0.028419,False,2017-03-222017-04-21
2017-03-22,MTUM2017-04-21p80.002017-03-22,MTUM,2017-04-21,0.70,1,0.00,0,0.00,put,80.0,0,0,82.34,30 days,-0.028419,False,2017-03-222017-04-21
2017-03-22,MTUM2017-04-21c81.002017-03-22,MTUM,2017-04-21,2.90,5,0.65,5,0.00,call,81.0,0,0,82.34,30 days,-0.016274,False,2017-03-222017-04-21
2017-03-22,MTUM2017-04-21p81.002017-03-22,MTUM,2017-04-21,1.70,5,0.00,0,0.00,put,81.0,0,0,82.34,30 days,-0.016274,False,2017-03-222017-04-21
2017-03-22,MTUM2017-04-21c82.002017-03-22,MTUM,2017-04-21,2.15,5,0.00,0,0.00,call,82.0,0,0,82.34,30 days,-0.004129,False,2017-03-222017-04-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-30,VOO2020-02-21P345.002019-12-30,VOO,2020-02-21,50.90,24,49.50,11,0,put,345.0,0,0,295.04,53 days,0.169333,False,2019-12-302020-02-21
2019-12-30,VOO2020-02-21P350.002019-12-30,VOO,2020-02-21,55.80,25,54.50,22,0,put,350.0,0,0,295.04,53 days,0.186280,False,2019-12-302020-02-21
2019-12-30,VOO2020-02-21P355.002019-12-30,VOO,2020-02-21,60.60,11,59.60,11,0,put,355.0,0,0,295.04,53 days,0.203227,False,2019-12-302020-02-21


In [46]:
mf = tf 
mf = mf.drop(columns=['DataDate'])
mf

,optionkey,Symbol,ExpirationDate,AskPrice,AskSize,BidPrice,BidSize,LastPrice,PutCall,StrikePrice,Volume,openinterest,UnderlyingPrice,time delta,% out,exp?,DateKey
DataDate,,,,,,,,,,,,,,,,,
2017-03-22,MTUM2017-04-21c80.002017-03-22,MTUM,2017-04-21,3.70,5,1.50,5,0.00,call,80.0,0,0,82.34,30 days,-0.028419,False,2017-03-222017-04-21
2017-03-22,MTUM2017-04-21p80.002017-03-22,MTUM,2017-04-21,0.70,1,0.00,0,0.00,put,80.0,0,0,82.34,30 days,-0.028419,False,2017-03-222017-04-21
2017-03-22,MTUM2017-04-21c81.002017-03-22,MTUM,2017-04-21,2.90,5,0.65,5,0.00,call,81.0,0,0,82.34,30 days,-0.016274,False,2017-03-222017-04-21
2017-03-22,MTUM2017-04-21p81.002017-03-22,MTUM,2017-04-21,1.70,5,0.00,0,0.00,put,81.0,0,0,82.34,30 days,-0.016274,False,2017-03-222017-04-21
2017-03-22,MTUM2017-04-21c82.002017-03-22,MTUM,2017-04-21,2.15,5,0.00,0,0.00,call,82.0,0,0,82.34,30 days,-0.004129,False,2017-03-222017-04-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-30,VOO2020-02-21P345.002019-12-30,VOO,2020-02-21,50.90,24,49.50,11,0,put,345.0,0,0,295.04,53 days,0.169333,False,2019-12-302020-02-21
2019-12-30,VOO2020-02-21P350.002019-12-30,VOO,2020-02-21,55.80,25,54.50,22,0,put,350.0,0,0,295.04,53 days,0.186280,False,2019-12-302020-02-21
2019-12-30,VOO2020-02-21P355.002019-12-30,VOO,2020-02-21,60.60,11,59.60,11,0,put,355.0,0,0,295.04,53 days,0.203227,False,2019-12-302020-02-21


In [8]:
###This isnt working

of = get_option_volume(tf, 'MTUM')
of

<ipython-input-6-10cbfc029784>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mf['Volume'] = mf['Volume'].astype(int)
<ipython-input-6-10cbfc029784>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mf['Total Volume'] = mf.groupby(['DataDate']).Volume.transform('sum')


ValueError: cannot reindex from a duplicate axis

In [6]:
### To slice the dataframe for a particular symbol or option type

def select_df(df, symbol, option):
    df = df.loc[df['PutCall'] == option]
    df = df.loc[df['Symbol'] == symbol]
    return df


###The first process, for the first features needed for the program

def feature_engineering(data):
    #for i in data:
    data['ExpirationDate'] = data['ExpirationDate'].apply(pd.Timestamp)
    data['DataDate'] = data['DataDate'].apply(pd.Timestamp)
    #data = data.loc[data['PutCall'] == 'put']

    #data['ExpirationDate'] =  pd.to_datetime(data['ExpirationDate'], infer_datetime_format=True)
    #data['DataDate'] =  pd.to_datetime(data['DataDate'], infer_datetime_format=True)
    data['time delta'] = data['ExpirationDate'] - data['DataDate']
    data['StrikePrice'] = pd.to_numeric(data['StrikePrice'])
    data['UnderlyingPrice'] = pd.to_numeric(data['UnderlyingPrice'])
    data['% out'] = data['StrikePrice'] / data['UnderlyingPrice'] - 1
    data['exp?'] = data['ExpirationDate'] == data['DataDate']
    #data['ExpirationDate']= data['ExpirationDate'].to_string
    #data['DataDate'] = data['DataDate'].to_string
    data['DateKey'] = data["DataDate"].astype(str) + data["ExpirationDate"].astype(str) 
    data = data.drop(columns=['DataDate'])

    #data['xp'] = data.loc[data['exp?'] == True, 'AskPrice']
    #data['xp']=data.OptionKey.map(xp.set_index('OptionKey')['xp'])

    return data



###To map the expiration value by optionkey, so it will appear on all 
###df.loc[df'AskPrice'] > df['xp']
def map_exp(df):
    xp = df
    xp.loc[xp['exp?']== True, 'xp' ] = xp.loc[df['exp?'] == True, 'AskPrice']
    xp = xp.loc[xp['exp?']== True]
    xp= xp[['optionkey', 'xp']]
    df['xp']=df.optionkey.map(xp.set_index('optionkey')['xp'])
    return df

###Pass the data frame with features in to this

def get_option_volume(option_df, symbol):
    #Calculate volume by date
    mf=option_df[option_df['Symbol']==symbol]
    #mf = mf.drop(columns=['DataDate'])

    mf['Volume'] = mf['Volume'].astype(int)
    mf['Total Volume'] = mf.groupby(['DataDate']).Volume.transform('sum')
    mf['Call Volume'] = mf.loc[mf['PutCall'] == 'call', 'Volume']
    mf['Put Volume'] = mf.loc[mf['PutCall'] == 'put', 'Volume']
    volume = mf.groupby('DataDate').agg({'Call Volume': 'sum', 'Put Volume' : 'sum'})
    volume['Total Volume'] = volume.loc[:,['Call Volume','Put Volume']].sum(axis=1)

    #Create put signals
    volume['Percent of Put'] = volume['Put Volume']/volume['Total Volume']
    volume['Put signals'] = volume['Percent of Put'] > .6

    #Create Call volume
    volume['Percent of calls'] = volume['Call Volume']/volume['Total Volume']
    volume['Call signals'] = volume['Percent of calls'] > .6
    volume_ratio = 3 #20
    volume['Option rVol'] = volume['Total Volume']/ volume['Total Volume'].rolling(window = volume_ratio).mean()
    

    volume['option rvol delta'] = volume['Option rVol'].dropna().pct_change()

    return volume


###Create the signals that will be passed to LSTM
###This result is supposed to work in LSTM, however it isnt right now


def get_signals(features):
    signals_df = pd.DataFrame()
    signals_df['volume delta'] = features['Volume'].dropna().pct_change()
    signals_df['bb std delta'] = features['bollinger_std'].dropna().pct_change()
    signals_df['rvol delta'] = features['rvol'].dropna().pct_change()
    signals_df['option rvol delta'] = features['Option rVol'].dropna().pct_change()
    signals_df['daily returns'] = features['daily returns'].dropna()

    return signals_df


### Pass the stock underlying dataframe in this


def get_underlying_volume(uf, symbol):

    bollinger_window=  20
    
    volume_df=uf[uf['Name']==symbol]
    #volume_df = volume_df.merge(volume, left_index=True, right_index = True)

     # more steps feature engineering
    volume_df.loc[:, 'bollinger_mid_band'] = volume_df['Close'].rolling(window=bollinger_window).mean()
    volume_df.loc[:, 'bollinger_std'] = volume_df['Close'].rolling(window=bollinger_window).std()
    
     #Calculate upper and lowers bands of bollinger band
     
    volume_df['bollinger_upper_band']  = volume_df['bollinger_mid_band'] + (volume_df['bollinger_std'] * 1)
    volume_df['bollinger_lower_band']  = volume_df['bollinger_mid_band'] - (volume_df['bollinger_std'] * 1)

     #Calculate bollinger band trading signal
    volume_df['bollinger_long'] = np.where(volume_df['Close'] < volume_df['bollinger_lower_band'], 1.0, 0.0)
    volume_df['bollinger_short'] = np.where(volume_df['Close'] > volume_df['bollinger_upper_band'], -1.0, 0.0)
    volume_df['bollinger_signal'] = volume_df['bollinger_long'] + volume_df['bollinger_short']
     #construct relative volume ratio and append to volume
    volume_ratio = 20

     #calculate rvol and 1 deviation of a 20 day rvol
    volume_df['rvol'] = volume_df['Volume']/ volume_df['Volume'].rolling(window = volume_ratio).mean()
    volume_df['rvol SD'] = volume_df['rvol'].rolling(window = volume_ratio).std()/2+ volume_df['rvol'].rolling(window = volume_ratio).mean()
#     #create rvol signal where rvol > 1 SD rvol
    volume_df['rvol signal'] = np.where(volume_df['rvol'] > volume_df['rvol SD'], 1.0, 0.0)

     #create options rVol signal
    volume_ratio = 20

     #create daily returns column and Actual Value column
    volume_df['daily returns'] = volume_df['Close'].dropna().pct_change()

     #drop nans due to BB/rvol calculations
    volume_df = volume_df.dropna()
   # return volume_df
    
    trading_signals = volume_df
    trading_signals['volume delta'] = volume_df['Volume'].dropna().pct_change()
    trading_signals['bb std delta'] = volume_df['bollinger_std'].dropna().pct_change()
    trading_signals['rvol delta'] = volume_df['rvol'].dropna().pct_change()
#     #add daily returns as target
    trading_signals['daily returns'] = volume_df['daily returns'].dropna()

    trading_signals= trading_signals.fillna(value = 0)
    trading_signals= trading_signals.replace([np.inf, -np.inf], 0.0)
    
    trading_signals.head()
    return trading_signals

###Merge the option and underlying volume dfs, and create extra signals

def merge(uv,ov):
    combined = uv.merge(ov, left_index = True, right_index = True, how = 'inner')
    volume_ratio = 3 #20
    combined['Option rVol SD'] = combined['rvol'].rolling(window = volume_ratio).std()/2+ combined['rvol'].rolling(window = volume_ratio).mean()

    combined['rvol signal'] = np.where(combined['rvol'] > combined['rvol SD'], 1.0, 0.0)

    combined['Option rVol signal'] = np.where(combined['rvol'] > combined['rvol SD'], 1.0, 0.0)
    return combined

#trading_signals = merge(underlying_volume, option_volume)

mf={}
def pipeline_features(ov, uv):
    for each_symbol in underlying['Name'].unique():
        ov = feature_engineering(ov)
        opt = get_option_volume(ov, each_symbol)
        undv = get_underlying_volume(uv, each_symbol)
        combined = merge(opt, undv)
        #lstm = lstm.main_lstm(combined)
        mf[each_symbol]=combined
        #save each of the features dataframes to csv
        #combined.to_csv(f'etf_{each_symbol}_initial.csv')
    return mf



In [52]:
##This is the code that needs to work

features = pipeline_features(df, underlying)
features

<ipython-input-51-10cbfc029784>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mf['Volume'] = mf['Volume'].astype(int)
<ipython-input-51-10cbfc029784>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mf['Total Volume'] = mf.groupby(['DataDate']).Volume.transform('sum')


ValueError: cannot reindex from a duplicate axis

In [ ]:
#This is eventually what the code will be
#Read the comments of the addition
#may want to add the additional testing elements
mf={}
def pipeline_features(ov, uv):
    for each_symbol in underlying['Name'].unique():
        ov = feature_engineering(ov)
        opt = get_option_volume(ov, each_symbol)
        undv = get_underlying_volume(uv, each_symbol)
        combined = merge(opt, undv)
        #Add the lstm function
        lstm_results = lstm.main_lstm(combined)
        #Add the rf function
        rf_results = rf.main_rf(combined)
        #Join the lstm signals to df
        #the lstm file may need to be altered so it doesn't drop any columns,
        #the format here will not be the same as the csv read before
        signal_ready = join_signals(df,lstm_results, rf_results)
        mf[each_symbol]= signal_ready
        #save each of the features dataframes to csv
        #combined.to_csv(f'etf_{each_symbol}_initial.csv')
    return mf


In [ ]:
##This might work

def join_signals(df, lstm_signal, rf_signal):
    lstm_signal = lstm_signal[['LSTM Predicted Signal']].copy()#add 'DataDate'
    rf_signal = rf_signal[['LSTM Predicted Signal']].copy()#add 'DataDate'

#     signal = signal.set_index(pd.to_datetime(signal.DataDate, infer_datetime_format=True))
    df = df.set_index(pd.to_datetime(df.DataDate, infer_datetime_format=True))

    first_result = df.merge(lstm_signal, left_index = True, right_index = True, how= 'inner')
    result = first_result.merge(rf_signal, left_index = True, right_index = True, how= 'inner')

    return result
    
    